<h1>Creating Graphs</h1>

In [ ]:
!pip install powerlaw

In [ ]:
import json

with open("/kaggle/input/cic-ids-20117/df_properties.json", "r") as f:
    df_properties = json.load(f)

print("📘 Dataset Properties:")
for k, v in df_properties.items():
    print(f"{k}: {v}")


In [ ]:
import pickle

with open("/kaggle/input/cic-ids-20117/labels_names.pkl", "rb") as f:
    labels_names = pickle.load(f)
print(labels_names)
print("\n🏷️ Label Class Mapping:")
# for k, v in labels_names.items():
#     print(f"{k}: {v}")


In [ ]:
# !pip install igraph
# !pip install networkx 

In [ ]:
import os
import pickle
import random
import socket
import struct

import sys
import pandas as pd
sys.path.append("/kaggle/input/gnn-nids/pytorch/default/1")

import networkx as nx
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.dataset.dataset_info import datasets
from src.graph.graph_construction import create_weightless_window_graph
from src.graph.graph_measures import calculate_graph_measures
from src.graph.centralities import add_centralities, add_centralities_as_node_features
# from local_variables import local_datasets_path
local_datasets_path = "/kaggle/input/"
original_datasets_files_path = "/kaggle/input/cic-ids-2017-gnn-2"
# /kaggle/input/cic-ids-2017-gnn-2


In [ ]:
multi_class = True

use_node_features = True    #------- False == No node feature used

use_port_in_address = False

generated_ips = False

graph_type = "flow"
# graph_type = "window"
# graph_type = "line"

# window_size= 400
window_size= 500
# print("1. flow sorted")
# print("2. flow using node features unsorted") 
# print("3. flow using node features sorted")
sort_timestamp = False      # --------- sorting timstamp

# k_fold = None
# k_fold = 5

validation_size = 0.1
test_size = 0.1

cn_measures = ["betweenness", "degree", "pagerank", "closeness", "k_truss"]
# cn_measures = ["betweenness", "degree", "closeness"]

network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank', 'src_closeness', 'dst_closeness', 'src_k_truss', 'dst_k_truss']
# network_features = ['src_betweenness', 'dst_betweenness', 'src_degree', 'dst_degree', 'src_pagerank', 'dst_pagerank']
GTP=graph_type+" "
if use_node_features:
    GTP+="using NODE FEATURES "
if sort_timestamp:
    GTP+="Sorted "
else:
    GTP+="Unsorted."
print(GTP)

In [ ]:
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
# name = "cic_ids_2017_5_percent"
name = "cic_ids_2017"
# name = "cic_bot_iot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"

dataset = datasets[name]

In [ ]:
g_type = ""
if graph_type == "flow":
    g_type = "flow"
elif graph_type == "line":
    g_type = f"line_graph_{window_size}"
elif graph_type == "window":
    g_type = f"window_graph_{window_size}"
    
if multi_class:
    g_type += "__multi_class"
    
if use_node_features:
    g_type += "__n_feats"
    
# if k_fold:
#     g_type += f"__{k_fold}_fold"
    
if use_port_in_address:
    g_type += "__ports"
    
if generated_ips:
    g_type += "__generated_ips"
    
if sort_timestamp:
    g_type += "__sorted"
else:
    g_type += "__unsorted"
# ****************** ALTERED    



dataset_path = os.path.join(local_datasets_path,name)
folder_path = os.path.join(dataset_path, g_type)
if name == "cic_ids_2017":
    name_ds = "cic-ids-2017-gnn-2"
    dataset_path = os.path.join(local_datasets_path,name_ds)
    # folder_path = os.path.join(local_datasets_path,name, g_type)
    folder_path= os.path.join("/kaggle/working/",name,g_type)

print(folder_path)
print(dataset_path)

In [ ]:
# if name== "cic_ids_2017":
#     df=pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))
# else:
df = pd.read_parquet(os.path.join(dataset_path, f"{name}.parquet"))

In [ ]:
df.head()

In [ ]:
cols_to_norm = list(set(list(df.columns))  - set(list([dataset.label_col, dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))

In [ ]:
df[dataset.label_col].value_counts()

In [ ]:
if generated_ips:
    df[dataset.src_ip_col] = df[dataset.src_ip_col].apply(lambda x: socket.inet_ntoa(struct.pack('>I', random.randint(0xac100001, 0xac1f0001))))

In [ ]:
if sort_timestamp:
    # df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)

In [ ]:
if use_port_in_address:
    df[dataset.src_port_col] = df[dataset.src_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.src_ip_col] = df[dataset.src_ip_col] + ':' + df[dataset.src_port_col]

    df[dataset.dst_port_col] = df[dataset.dst_port_col].astype(float).astype(int).astype(str) # to remove the decimal point
    df[dataset.dst_ip_col] = df[dataset.dst_ip_col] + ':' + df[dataset.dst_port_col]

In [ ]:
df.head(20)

In [ ]:
print(dataset.class_num_col)

In [ ]:
if multi_class:
    y = df[dataset.class_num_col]
else:
    y = df[dataset.label_col]

if sort_timestamp:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size)
else:
    X_tr, X_test, y_tr, y_test = train_test_split(
        df, y, test_size=test_size, random_state=13, stratify=y)
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_tr, y_tr, test_size=validation_size, random_state=13, stratify=y_tr)

del df

In [ ]:
if graph_type == "line" and use_node_features:
    add_centralities(df = X_train, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_val, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    add_centralities(df = X_test, new_path=None, graph_path=None, dataset=dataset, cn_measures=cn_measures, network_features=network_features, create_using=nx.MultiDiGraph())
    cols_to_norm = list(set(cols_to_norm) | set(network_features))
    

In [ ]:
scaler = StandardScaler()

X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
X_train['h'] = X_train[ cols_to_norm ].values.tolist()

cols_to_drop = list(set(list(X_train.columns)) - set(list([dataset.label_col, dataset.src_ip_col, dataset.dst_ip_col, dataset.class_num_col, 'h'])))
X_train.drop(cols_to_drop, axis=1, inplace=True)

X_val[cols_to_norm] = scaler.transform(X_val[cols_to_norm])
X_val['h'] = X_val[ cols_to_norm ].values.tolist()
X_val.drop(cols_to_drop, axis=1, inplace=True)

X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test[ cols_to_norm ].values.tolist()
X_test.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
if graph_type == "window" or graph_type == "line":

    create_weightless_window_graph(
        df=X_train,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "training"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_val,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "validation"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")
    
    create_weightless_window_graph(
        df=X_test,
        dataset=dataset,
        window_size=window_size,
        line_graph=graph_type == "line",
        folder_path=os.path.join(folder_path, "testing"),
        edge_attr= ['h', dataset.label_col, dataset.class_num_col],
        file_type="pkl")

In [ ]:
if graph_type == "flow":
	os.makedirs(folder_path, exist_ok=True)
	print(f"==>> X_train.shape: {X_train.shape}")
	print(f"==>> X_val.shape: {X_val.shape}")
	print(f"==>> X_test.shape: {X_test.shape}")


In [ ]:
if graph_type == "flow":
    graph_name = "training_graph"

    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

In [ ]:
if graph_type == "flow":
    graph_name = "validation_graph"

    G = nx.from_pandas_edgelist(X_val, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")

    # graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")

    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

In [ ]:
if graph_type == "flow":
    graph_name = "testing_graph"
    
    G = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h', dataset.label_col, dataset.class_num_col], create_using=nx.MultiDiGraph())
    
    if use_node_features:
        add_centralities_as_node_features(df=None, G=G, graph_path=None, dataset=dataset, cn_measures=cn_measures)
        
        for node in G.nodes():
            centralities = []
            for centrality in cn_measures:
                centralities.append(G.nodes[node].get(centrality, 0)) # Default to 0 if missing
                
                # Combine features into a single vector
            n_feats = np.array(centralities, dtype=np.float32)
            
            # Add the new feature to the node
            G.nodes[node]["n_feats"] = n_feats
            
    graph_measures = calculate_graph_measures(G, f"{folder_path}/{graph_name}_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    # graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    # print(f"==>> graph_measures: {graph_measures}")
    
    with open(f"{folder_path}/{graph_name}.pkl", "wb") as f:
        pickle.dump(G, f)

In [ ]:
# Save all installed package versions
!pip freeze > requirements_1_GNN.txt

# Display first few lines
!head -n 20 requirements.txt


In [ ]:
import json

print("-------------------------- SAVING VARIABLES -------------------------- ")
data = {
    "dataset_path": dataset_path,
    "folder_path": folder_path,
    "name": name,
    "g_type": g_type
}

# Save to JSON
with open("config.json", "w") as f:
    json.dump(data, f, indent=4)


In [ ]:
import psutil
import gc
ram_gb = psutil.virtual_memory().used / 1e9
print(f"Current RAM usage: {ram_gb:.2f} GB")
gc.collect()
print("-------------------------- CLEARING THE MEMORY -------------------------- ")
print("Clear all variables (like restarting kernel)")
%reset -f


In [ ]:
import psutil
ram_gb = psutil.virtual_memory().used / 1e9
print(f"Current RAM usage: {ram_gb:.2f} GB")

<h1>TRAINNING</h1>

In [ ]:
import json
print("-------------------------start Training-----------------------------")
print("-------------------------- RETRIVING VARIABLES --------------------------")
with open("config.json", "r") as f:
    config = json.load(f)

# Access the values
dataset_path = config["dataset_path"]
folder_path = config["folder_path"]
name = config["name"]
g_type = config["g_type"]


In [ ]:
print(folder_path)

In [ ]:

# print(folder_path)
# print(g_type)
# dataset_name = "cic_ids_2017"
# # dataset_folder = "/kaggle/input/cic-ids-2017-gnn"
# dataset_folder = "/kaggle/input/cic-ids-2017-gnn-2"
# # grpah_folder="/kaggle/input/cic-ids-2017-gnnflow-multi-class-unsorted"
# graphs_folder=folder_path
# # /kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted
# # "/kaggle/input/cic-ids-2017-gnn"
# # /kaggle/input/d/mohammadfleity/cic-ids-2017-gnn/cic_ids_2017.parquet
# model_path = "/kaggle/input/gnn-nids-wandb/pytorch/default/6/"
dataset_name = name
dataset_folder = dataset_path
graph_folder=folder_path
model_path = "/kaggle/input/gnn-nids-wandb/pytorch/default/6/"
using_wandb = False
save_top_k = 1


# save_top_k = 4 save best 4 test model from the traininng based on f1_score 
# early_stopping_patience = max_epochs = 20
early_stopping_patience = max_epochs = 500
# early_stopping_patience = 200 all - epochs = 500 - lr = 0.005
learning_rate = 0.005
weight_decay = 0.01
ndim_out = [32, 32]
num_layers = 2
number_neighbors = [25, 10]
dropout = 0.5
residual = True
multi_class = True
aggregation = "mean"
# aggregation = "sum"

use_centralities_nfeats = False

# g_type = "flow"

# if multi_class:
#     g_type += "__multi_class"

# if use_centralities_nfeats:
#     g_type += "__n_feats"
    
# if sort_timestamp:
#     g_type += "__sorted"
# else:
#     g_type += "__unsorted"

In [ ]:
pip uninstall -y torch torchvision torchaudio torchdata pytorch-lightning --quiet

In [ ]:
print("dgl")


In [ ]:
# pip install dgl 

In [ ]:
pip install dgl -f https://data.dgl.ai/wheels/torch-2.1/cu118/repo.html 

In [ ]:
print("done_dgl")

In [ ]:
pip install torch==2.1.0+cu118 torchvision==0.16.0+cu118 torchaudio==2.1.0+cu118 -f https://download.pytorch.org/whl/torch_stable.html --quiet

In [ ]:
import torch
print("torch version:", torch.__version__)
print("CUDA available?", torch.cuda.is_available())

In [ ]:
!apt update -qq 2>/dev/null >/dev/null;
!apt install -y libcusparse11 -qq 2>/dev/null >/dev/null;


In [ ]:
pip install --upgrade -q pytorch-lightning==2.5.0

In [ ]:
!pip install --upgrade -q wandb==0.19.6

In [ ]:
# import wandb
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("mohammad_wandb_secret")

wandb.login(key=secret_value_0)

In [ ]:
import os
import json
import pickle
import time
import timeit
import importlib.util
import sys
import random

import numpy as np

os.environ["DGLBACKEND"] = "pytorch"

import torch.nn as nn
import torch
import warnings
import wandb
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

seed = 42  # or any constant value
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

run_dtime = time.strftime("%Y%m%d-%H%M%S")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# pip install torchdata

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
# import kagglehub

# # Download latest version
# path = kagglehub.model_download("mortadaphdtermos/gnn-nids-wandb/pyTorch/default")

# print("Path to model files:", path)

In [ ]:
# pip install torchdata==0.7.0

In [ ]:
# pip install torch.utils

In [ ]:
def add_lib(module_name, path):
    spec = importlib.util.spec_from_file_location(module_name, path)
    dataset_info = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = dataset_info
    spec.loader.exec_module(dataset_info)

add_lib("dataset_info", model_path + "dataset_info.py")
add_lib("lightning_data", model_path + "lightning_data.py")
add_lib("lightning_model", model_path + "lightning_model.py")
add_lib("models", model_path + "models.py")

In [ ]:
from models import EGAT, EGCN, EGRAPHSAGE
from lightning_model import GraphModel
from lightning_data import GraphDataModule
from dataset_info import datasets
import time
import psutil
import pandas as pd
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [ ]:
# pip install torchmetrics

In [ ]:
# pip install torch_geometric

In [ ]:
import threading, time, psutil

try:
    import pynvml
    pynvml.nvmlInit()
    GPU_AVAILABLE = True
except:
    GPU_AVAILABLE = False

def monitor_resources(process, peaks, interval=0.2):
    """Monitor CPU + GPU memory usage in a background thread."""
    peaks["cpu"] = 0.0
    peaks["gpu"] = 0.0
    
    handle = None
    if GPU_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    
    while getattr(threading.current_thread(), "running", True):
        try:
            cpu_mem = process.memory_info().rss / (1024 ** 2)
            gpu_mem = 0.0
            if GPU_AVAILABLE:
                gpu_mem = pynvml.nvmlDeviceGetMemoryInfo(handle).used / (1024 ** 2)
            peaks["cpu"] = max(peaks["cpu"], cpu_mem)
            peaks["gpu"] = max(peaks["gpu"], gpu_mem)
        except psutil.NoSuchProcess:
            break
        time.sleep(interval)


<h2>E_GAT</h2>

In [ ]:
# pip install _XLAC

In [ ]:
# import torch_xla.core.xla_model as xm # FOR TPU ONLY

# # For XLA tensors
# xla_tensor = self.graph.edata[self.class_num_col]
# target = xm.to_cpu(xla_tensor).detach().numpy()


In [ ]:
import os
import pickle
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger
import numpy as np
import wandb
import csv

In [ ]:
# Save all installed package versions
!pip freeze > requirements_2_GNN.txt

# Display first few lines
!head -n 20 requirements.txt


In [ ]:
warnings.filterwarnings("ignore", ".*does not have many workers.*")

dataset = datasets[dataset_name]

# Hyperparameters
aggregation = "mean"
activation = F.relu

graphs_folder = folder_path
# if dataset_name == "cic_ids_2017":
    # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__sorted"

logs_folder = os.path.join("logs", dataset.name)
os.makedirs(logs_folder, exist_ok=True)

wandb_runs_path = os.path.join("logs", "wandb_runs")
os.makedirs(wandb_runs_path, exist_ok=True)

labels_mapping = {0: "Normal", 1: "Attack"}
num_classes = 2
if multi_class:
    with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
        labels_names = pickle.load(f)
    labels_mapping = labels_names[0]
    num_classes = len(labels_mapping)

dataset_kwargs = dict(
    use_node_features=use_centralities_nfeats,
    multi_class=True,
    using_masking=False,
    masked_class=2,
    num_workers=0,
    label_col=dataset.label_col,
    class_num_col=dataset.class_num_col,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

data_module = GraphDataModule(graphs_folder, batch_size=1, **dataset_kwargs)
data_module.setup()

ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
edim = next(iter(data_module.train_dataloader())).edata["h"].shape[-1]

my_models = {
    "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes),
    f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,residual, num_classes, num_neighbors=None, aggregation=aggregation),
    f"e_graphsage_{aggregation}": EGRAPHSAGE(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors,
        aggregation=aggregation
    ),
    f"e_graphsage_sum_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,residual, num_classes, num_neighbors=None, aggregation="sum"),
    f"e_graphsage_sum": EGRAPHSAGE(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors,
        aggregation="sum"
    ),
    "e_gat_sampling": EGAT(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=number_neighbors
    ),
    "e_gat_no_sampling": EGAT(
        ndim, edim, ndim_out, num_layers, activation, dropout,
        residual, num_classes, num_neighbors=None
    ),
}

criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

for model_name, model in my_models.items():

    config = {
        "run_dtime": run_dtime,
        "type": "GNN",
        "model_name": model_name,
        "max_epochs": max_epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "ndim_out": ndim_out,
        "num_layers": num_layers,
        "number_neighbors": number_neighbors,
        "activation": activation.__name__,
        "dropout": dropout,
        "residual": residual,
        "multi_class": multi_class,
        "aggregation": aggregation,
        "early_stopping_patience": early_stopping_patience,
        "use_centralities_nfeats": use_centralities_nfeats,
    }

    graph_model = GraphModel(
        model, criterion, learning_rate, config, model_name,
        labels_mapping, weight_decay=weight_decay,
        using_wandb=using_wandb, norm=False, multi_class=True
    )

    if using_wandb:
        wandb_logger = WandbLogger(
            project=f"GNN-Analysis-{dataset.name}",  # Project Name Wandb
            name=f"{model_name}---{g_type}",
            config=config,
            save_dir=wandb_runs_path,
        )
    else:
        wandb_logger = None

    f1_checkpoint_callback = ModelCheckpoint(
        monitor="val_f1_score",
        mode="max",
        filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
        save_top_k=save_top_k,
        verbose=False,
    )

    early_stopping_callback = EarlyStopping(
        monitor="val_loss",
        mode="min",
        patience=early_stopping_patience,
        verbose=False,
    )
    trainer = pl.Trainer(
        max_epochs=max_epochs,
        num_sanity_val_steps=0,
        log_every_n_steps=0,
        callbacks=[f1_checkpoint_callback, early_stopping_callback],
        default_root_dir=logs_folder,
        logger=wandb_logger if using_wandb else None,
    )
    # start_time = time.time()
    # process = psutil.Process()
    print("------------- start training ---------------------")
    
    # Measure start time and memory
    cpu_mem_used=gpu_mem_used=elapsed_time=0
    process = psutil.Process(os.getpid())
    cpu_mem_before = process.memory_info().rss / (1024 ** 2)
    handle = None
    if GPU_AVAILABLE:
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    if GPU_AVAILABLE:
        gpu_mem_before = pynvml.nvmlDeviceGetMemoryInfo(handle).used / (1024 ** 2)
    
    process = psutil.Process(os.getpid())
    peaks = {}
    
    monitor_thread = threading.Thread(target=monitor_resources, args=(process, peaks))
    monitor_thread.running = True
    monitor_thread.start()
    
    start_time = time.time()
    
    trainer.fit(graph_model, datamodule=data_module)
    
    test_results = []
    print(process)
    for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):      # ---- currently 1 model is saved only 
        graph_model.test_prefix = f"best_f1_{i}"
        results = trainer.test(graph_model, datamodule=data_module, ckpt_path=k)
        # End time and memory
        elapsed_time = time.time() - start_time
        # Stop thread safely
        monitor_thread.running = False
        monitor_thread.join()
        
        peak_cpu = peaks.get("cpu", 0.0)
        peak_gpu = peaks.get("gpu", 0.0)
        cpu_mem_used=peak_cpu-cpu_mem_before
        gpu_mem_used=peak_gpu-gpu_mem_before
        
        test_results.append(results[0][f"best_f1_{i}_test_f1"])
        # End time & memory
        print(f"\n--- Test Run {i+1} ---")
        print("graph_model: ", graph_model)
        print("data_module: ", data_module)
        print("results: ", results)
        logs = {
        "median_f1_of_best_f1": np.median(test_results),
        "max_f1_of_best_f1": np.max(test_results),
        "avg_f1_of_best_f1": np.mean(test_results),
            
    }
    print("test_results: ", test_results)

    metric_dir = "/kaggle/working/temp"
    os.makedirs(metric_dir, exist_ok=True)
    
    json_path = os.path.join(metric_dir, f"{model_name}_metrics.json")
    
    # Create dataframe with a single row
    df = pd.DataFrame([{
        # "model_name": model_name,
        "TEST_ACCURACY": test_results[0],
        "Time Consumption (s)": elapsed_time,
        # "Memory Consumption (MB)": memory_consumption_mb
        "CPU_Peak_MB": peak_cpu,
        "GPU_Peak_MB": peak_gpu,
        "cpu_mem_used":cpu_mem_used,
        "gpu_mem_used":gpu_mem_used,
    }])
    
    # Save to JSON (records = list of dicts)
    df.to_json(json_path, orient="records", indent=4)
    
    print(f"Metrics saved to {json_path}")


    if using_wandb:
        wandb.log(logs)
        wandb.finish()
    else:
        trainer.logger.log_metrics(logs, step=trainer.global_step)


In [ ]:
import os
import json
import pandas as pd

# Your label mapping (from the print output you shared)
labels_map = {
    "0": "BENIGN",
    "1": "Bot",
    "2": "DDoS",
    "3": "DoS GoldenEye",
    "4": "DoS Hulk",
    "5": "DoS Slowhttptest",
    "6": "DoS slowloris",
    "7": "FTP-Patator",
    "8": "Heartbleed",
    "9": "Infiltration",
    "10": "PortScan",
    "11": "SSH-Patator",
    "12": "Web Attack \ufffd Brute Force",
    "13": "Web Attack \ufffd Sql Injection",
    "14": "Web Attack \ufffd XSS"
}
def json_to_csv_per_file(json_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    rows_all=[]
    rows_m_avg=[]
    for filename in os.listdir(json_folder):
        if filename.endswith("_results.json"):
            no_ext = filename.replace("_results.json", "")
            # Remove the trailing "_metrics"
            print(no_ext)
            model_name=no_ext
            metric_path = os.path.join(json_folder,f"{no_ext}_metrics.json")
            print(metric_path)
            filepath = os.path.join(json_folder, filename)
            with open(filepath, "r") as f:
                data = json.load(f)
            rows = []
            # --- Per-class metrics ---
            classification_report=data["classification_report"]
            # print(classification_report)
            if "results_fpr_fnr" in data and "per_class" in data["results_fpr_fnr"]:
                # print(data["results_fpr_fnr"])
                for cls, metrics in data["results_fpr_fnr"]["per_class"].items():
                    attack=labels_map.get(cls,cls)
                    print(f"{attack}")
                    metrics_d=classification_report.get(attack)
                    row = {
                        "ATTACK TYPE": attack,
                        "precision": metrics_d.get("precision"),
                        "recall": metrics_d.get("recall"),
                        "f1_score": metrics_d.get("f1-score"),
                        "support": metrics_d.get("support"),
                        "FPR": metrics.get("FPR"),
                        "FNR": metrics.get("FNR"),
                    }
                    # print(row)
                    # print(metrics_d.get("precision"))
                    # print()
                    rows.append(row)

            df_classes = pd.DataFrame(rows)
            weighted_avg=classification_report['weighted avg']
            macro_avg=classification_report['macro avg']
            # print(data)
            # print(macro_avg)
            # print(data['classification_report']['accuracy'])
            # --- Global / test metrics ---
            with open(metric_path, "r") as f:
                device_data = json.load(f)
            print(device_data)
            elapsed_time=device_data[0]["Time Consumption (s)"]
            peak_cpu=device_data[0]["CPU_Peak_MB"]
            peak_gpu=device_data[0]["GPU_Peak_MB"]
            cpu_mem_used=device_data[0]["cpu_mem_used"]
            gpu_mem_used=device_data[0]["gpu_mem_used"]
            # print("Memory:", MEMORY_USED)
            # print("Time:", TIME_CONSUMED)
            global_metrics = {
                'model_name':model_name,
                "test_weighted_f1": data.get("test_weighted_f1"),
                'test_accuracy':data['classification_report']['accuracy'],
                'precision': weighted_avg['precision'],
                'recall': weighted_avg['recall'], 
                'support': weighted_avg['support'],
                # "test_fpr": data.get("test_fpr"),
                # "test_fnr": data.get("test_fnr"),
                "global_FPR": data["results_fpr_fnr"]["global"].get("FPR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                "global_FNR": data["results_fpr_fnr"]["global"].get("FNR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                # "time_consumption":TIME_CONSUMED,
                # "memory_consumption":MEMORY_USED,
                "Time Consumption (s)": elapsed_time,
                "CPU_Peak_MB": peak_cpu,
                "GPU_Peak_MB": peak_gpu,
                "cpu_mem_used":cpu_mem_used,
                "gpu_mem_used":gpu_mem_used,
            }
            rows_all.append(
                {
                'model_name':model_name,
                "test_weighted_f1": data.get("test_weighted_f1"),
                'test_accuracy':data['classification_report']['accuracy'],
                'precision': weighted_avg['precision'],
                'recall': weighted_avg['recall'], 
                'support': weighted_avg['support'],
                # "test_fpr": data.get("test_fpr"),
                # "test_fnr": data.get("test_fnr"),
                "global_FPR": data["results_fpr_fnr"]["global"].get("FPR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                "global_FNR": data["results_fpr_fnr"]["global"].get("FNR") if "results_fpr_fnr" in data and "global" in data["results_fpr_fnr"] else None,
                # "time_consumption":TIME_CONSUMED,
                # "memory_consumption":MEMORY_USED,
                "Time Consumption (s)": elapsed_time,
                "CPU_Peak_MB": peak_cpu,
                "GPU_Peak_MB": peak_gpu,
                "cpu_mem_used":cpu_mem_used,
                "gpu_mem_used":gpu_mem_used,
                }
            )
            print(global_metrics)
            # {'precision': 0.4393356617629928, 'recall': 0.5886290738703498, 'f1-score': 0.3932640639233801, 'support': 228621}
            macro_metrics = {
                'model_name':model_name,
                'f1-score': macro_avg['f1-score'],
                'precision': macro_avg['precision'],
                'recall': macro_avg['recall'], 
                'support': macro_avg['support'],
            }
            rows_m_avg.append(
                {
                'model_name':model_name,
                'f1-score': macro_avg['f1-score'],
                'precision': macro_avg['precision'],
                'recall': macro_avg['recall'], 
                'support': macro_avg['support'],
            }
            )
            print(macro_metrics)
            df_global = pd.DataFrame([global_metrics])
            # --- Save CSV ---
            csv_output = os.path.join(output_folder, filename.replace(".json", ".csv"))
            with open(csv_output, "w") as f:
                f.write("### Per-class Metrics\n")
            df_classes.to_csv(csv_output, mode="a", index=False)
            with open(csv_output, "a") as f:
                f.write("\n### Global Metrics\n")
            df_global.to_csv(csv_output, mode="a", index=False)

            print(f"Saved {csv_output}")
        df = pd.DataFrame(rows_all)
    temp_dir = "/kaggle/working/temp/csv_results/"
    # print(folder_path)
    folder=os.path.join("/kaggle/working/",dataset_name)
    graph_title=folder_path.replace(f"{folder}/","")
    # print(graph_title)
    # Create folder if it doesn't exist
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    
    # Save CSV inside temp/
    if dataset_name=="cic_ton_iot":
        csv_path = os.path.join(temp_dir, f"{graph_title}__GNN_BENCHMARKING_CIC_TON_IOT.csv")
    else:
        csv_path = os.path.join(temp_dir, f"{graph_title}__GNN_BENCHMARKING_CIC_IDS_2017.csv")
    df.to_csv(csv_path, index=False)
    print(f"✅ CSV saved as {csv_path}")
    
    df = pd.DataFrame(rows_m_avg)
    temp_dir = "/kaggle/working/temp/csv_results/"
    
    # Create folder if it doesn't exist
    if not os.path.exists(temp_dir):
        os.makedirs(temp_dir)
    
    # Save CSV inside temp/
    if dataset_name=="cic_ton_iot":
        csv_path = os.path.join(temp_dir, f"{graph_title}__GNN_macro_results_CIC_TON_IOT.csv")
    else:
        csv_path = os.path.join(temp_dir, f"{graph_title}__GNN_macro_results_CIC_IDS_2017.csv")
    df.to_csv(csv_path, index=False)
    print(f"✅ CSV saved as {csv_path}")

json_folder = "/kaggle/working/temp/"
output_folder = "/kaggle/working/temp/csv_results"
json_to_csv_per_file(json_folder, output_folder)



In [ ]:
print(folder_path)
folder=os.path.join("/kaggle/working/",dataset_name)
graph_title=folder_path.replace(f"{folder}/","")
print(graph_title)
# /kaggle/working/cic_ids_2017/

<h2>E_GAT Sampling</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
# #     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE mean</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
# #     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                                          residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #               residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE sum</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="sum"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
#     # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                                          residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #               residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE Sampling mean</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="mean"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
#     # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                              residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>EGraphSAGE Sampling sum</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="sum"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
#     # graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     # "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#     #               dropout, residual, num_classes),
#     f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#                                              residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)

<h2>E_GCN</h2>

In [ ]:
# warnings.filterwarnings("ignore", ".*does not have many workers.*")

# dataset = datasets[dataset_name]

# # Hyperparameters
# aggregation="mean"

# activation = F.relu
# # graphs_folder = os.path.join(dataset_folder, g_type)
# # if dataset_name == "cic_ids_2017":
# #     graphs_folder = "/kaggle/working/cic_ids_2017/flow__multi_class__n_feats__unsorted"
# logs_folder = os.path.join("logs", dataset.name)
# os.makedirs(logs_folder, exist_ok=True)
# wandb_runs_path = os.path.join("logs", "wandb_runs")
# os.makedirs(wandb_runs_path, exist_ok=True)

# labels_mapping = {0: "Normal", 1: "Attack"}
# num_classes = 2
# if multi_class:
#     # with open(os.path.join(dataset_folder, "labels_names.pkl"), "rb") as f:
#     # /kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl
#     with open("/kaggle/input/cic-ids-2017-gnn-2/labels_names.pkl", "rb") as f:
#         labels_names = pickle.load(f)
#     labels_mapping = labels_names[0]
# num_classes = len(labels_mapping)

# dataset_kwargs = dict(
#     use_node_features=use_centralities_nfeats,
#     multi_class=True,
#     using_masking=False,
#     masked_class=2,
#     num_workers=0,
#     label_col=dataset.label_col,
#     class_num_col=dataset.class_num_col,
#     device='cuda' if torch.cuda.is_available() else "cpu"
# )

# data_module = GraphDataModule(
#     graphs_folder, batch_size=1, **dataset_kwargs)
# data_module.setup()

# ndim = next(iter(data_module.train_dataloader())).ndata["h"].shape[-1]
# edim = next(iter(data_module.train_dataloader())).edata['h'].shape[-1]

# my_models = {
#     "e_gcn": EGCN(ndim, edim, ndim_out, num_layers, activation,
#                   dropout, residual, num_classes),
#     # f"e_graphsage_{aggregation}": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                          residual, num_classes, num_neighbors=number_neighbors, aggregation=aggregation),
#     # f"e_graphsage_{aggregation}_no_sampling": EGRAPHSAGE(ndim, edim, ndim_out, num_layers, activation, dropout,
#     #                                                      residual, num_classes, num_neighbors=None, aggregation=aggregation),
#     # "e_gat_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout,
#                   # residual, num_classes, num_neighbors=number_neighbors),
#     # "e_gat_no_sampling": EGAT(ndim, edim, ndim_out, num_layers, activation, dropout, residual, num_classes, num_neighbors=None),
# }

# criterion = nn.CrossEntropyLoss(data_module.train_dataset.class_weights)

# for model_name, model in my_models.items():
        
#     config = {
#         "run_dtime": run_dtime,
#         "type": "GNN",
#         "model_name": model_name,
#         "max_epochs": max_epochs,
#         "learning_rate": learning_rate,
#         "weight_decay": weight_decay,
#         "ndim_out": ndim_out,
#         "num_layers": num_layers,
#         "number_neighbors": number_neighbors,
#         "activation": activation.__name__,
#         "dropout": dropout,
#         "residual": residual,
#         "multi_class": multi_class,
#         "aggregation": aggregation,
#         "early_stopping_patience": early_stopping_patience,
#         "use_centralities_nfeats": use_centralities_nfeats,}

#     graph_model = GraphModel(model, criterion, learning_rate, config, model_name,
#                              labels_mapping, weight_decay=weight_decay, using_wandb=using_wandb, norm=False, multi_class=True)

    
#     if using_wandb:
#         wandb_logger = WandbLogger(
#             project=f"GNN-Analysis-{dataset.name}", # Project Name Wandb
#             name=f"{model_name}---{g_type}",
#             config=config,
#             save_dir=wandb_runs_path
#         )
#     else:
#         wandb_logger = None
        
#     f1_checkpoint_callback = ModelCheckpoint(
#         monitor="val_f1_score",
#         mode="max",
#         filename="best-val-f1-{epoch:02d}-{val_f1_score:.2f}",
#         save_top_k=save_top_k,
#         verbose=False,
#     )
#     early_stopping_callback = EarlyStopping(
#         monitor="val_loss",
#         mode="min",
#         patience=early_stopping_patience,
#         verbose=False,
#     )

#     trainer = pl.Trainer(
#         max_epochs=max_epochs,
#         num_sanity_val_steps=0,
#         log_every_n_steps=0,
#         callbacks=[
#             f1_checkpoint_callback,
#             early_stopping_callback
#         ],
#         default_root_dir=logs_folder,
#         logger=wandb_logger if using_wandb else None,
#     )

#     trainer.fit(graph_model, datamodule=data_module)
    
#     test_results = []
    
#     for i, k in enumerate(f1_checkpoint_callback.best_k_models.keys()):
#         graph_model.test_prefix = f"best_f1_{i}"
#         results = trainer.test(
#             graph_model, datamodule=data_module, ckpt_path=k)
#         test_results.append(results[0][f"best_f1_{i}_test_f1"])

#     logs = {
#         "median_f1_of_best_f1": np.median(test_results),
#         "max_f1_of_best_f1": np.max(test_results),
#         "avg_f1_of_best_f1": np.mean(test_results)
#     }

#     if using_wandb:
#         wandb.log(logs)
#         wandb.finish()
#     else:
#         trainer.logger.log_metrics(logs, step=trainer.global_step)